Reference link: https://keras.io/examples/nlp/lstm_seq2seq/

Character level recurrent sequence to sequence model: Encoder Decoder Architecture

**Summary of the algorithm**

- We start with input sequences from a domain and corresponding target sequences from another domain.
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

**Tweaks done in the code from our side:**

1. The input format example given in our example (reference link) was quite different from the one that our polynomial data provides. Tweaking the input type in a way that the model can process is one of the major changes done to the code. 

2. Previously I also tried fitting an simple LSTM Seq to Seq model which gave us an encoding issue and an accuracy of around 40%. This issue was fixed in our current model where the encoding is done on a character level which gives us the desirable output. 

3. The given load file method in main was not working and hence I have switched to loading data in a different way from the URL. 

## Importing libraries


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


## Configuration


In [5]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 700_000  # Number of samples to train on.


In [1]:
import sys
import numpy as np
import urllib.request
from typing import Tuple

MAX_SEQUENCE_LENGTH = 29
TRAIN_URL = "https://scale-static-assets.s3-us-west-2.amazonaws.com/ml-interview/expand/train.txt"

def load_file(file_path: str) -> Tuple[Tuple[str], Tuple[str]]:
    """ A helper functions that loads the file into a tuple of strings

    :param file_path: path to the data file
    :return factors: (LHS) inputs to the model
            expansions: (RHS) group truth
    """
    with urllib.request.urlopen(file_path) as url:
        data = url.readlines()
        factors, expansions = zip(*[line.decode('utf8').strip().split("=") for line in data])
        return factors, expansions

In [2]:
factors, expansions = load_file(TRAIN_URL)

In [3]:
text_pairs = []
for i in range(len(factors)):
    fct = factors[i]
    exp = expansions[i]
    text_pairs.append((fct,exp))

## Prepare the data


In [7]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in text_pairs[: min(num_samples, len(text_pairs) - 1)]:
    input_text, target_text = line
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n" + " "
    input_text = input_text + " "
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 700000
Number of unique input tokens: 29
Number of unique output tokens: 31
Max sequence length for inputs: 30
Max sequence length for outputs: 31


## Build the model


In [8]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [9]:
# Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 29)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 31)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        292864      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

## Train the model


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=100,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/100
2432/2432 [==============================] - 31s 11ms/step - loss: 0.3011 - accuracy: 0.8904 - val_loss: 0.2264 - val_accuracy: 0.9168
Epoch 2/100
2432/2432 [==============================] - 26s 11ms/step - loss: 0.1800 - accuracy: 0.9313 - val_loss: 0.1554 - val_accuracy: 0.9402
Epoch 3/100
2432/2432 [==============================] - 27s 11ms/step - loss: 0.1192 - accuracy: 0.9534 - val_loss: 0.1053 - val_accuracy: 0.9593
Epoch 4/100
2432/2432 [==============================] - 26s 11ms/step - loss: 0.0812 - accuracy: 0.9681 - val_loss: 0.0770 - val_accuracy: 0.9704
Epoch 5/100
2432/2432 [==============================] - 25s 10ms/step - loss: 0.0619 - accuracy: 0.9749 - val_loss: 0.0630 - val_accuracy: 0.9744
Epoch 6/100
2432/2432 [==============================] - 26s 11ms/step - loss: 0.0525 - accuracy: 0.9783 - val_loss: 0.0634 - val_accuracy: 0.9737
Epoch 7/100
2432/2432 [==============================] - 26s 11ms/step - loss: 0.0460 - accuracy: 0.9808 - val_loss: 0

## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [ ]:
! unzip '/content/s2s.zip'

Archive:  /content/s2s.zip
   creating: s2s/assets/
  inflating: s2s/variables/variables.index  
  inflating: s2s/keras_metadata.pb   
  inflating: s2s/saved_model.pb      
  inflating: s2s/variables/variables.data-00000-of-00001  


In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("/content/s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



You can now generate decoded sentences as such:


In [ ]:
count = 0
total = 10
for seq_index in range(total):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence.strip())
    print("Target sentence:", target_texts[seq_index].strip())

-
Input sentence: (7-3*z)*(-5*z-9) 
Decoded sentence: 15*z**2-8*z-63
Target sentence: 15*z**2-8*z-63
-
Input sentence: -9*s**2 
Decoded sentence: -9*s**2
Target sentence: -9*s**2
-
Input sentence: (2-2*n)*(n-1) 
Decoded sentence: -2*n**2+4*n-2
Target sentence: -2*n**2+4*n-2
-
Input sentence: x**2 
Decoded sentence: x**2
Target sentence: x**2
-
Input sentence: (4-x)*(x-23) 
Decoded sentence: -x**2+27*x-92
Target sentence: -x**2+27*x-92
-
Input sentence: (7-5*c)*(3*c-17) 
Decoded sentence: -15*c**2+106*c-119
Target sentence: -15*c**2+106*c-119
-
Input sentence: -8*x*(3*x+14) 
Decoded sentence: -24*x**2-112*x
Target sentence: -24*x**2-112*x
-
Input sentence: -2*k*(5*k-9) 
Decoded sentence: -10*k**2+18*k
Target sentence: -10*k**2+18*k
-
Input sentence: (3*cos(c)-19)*(7*cos(c)+13) 
Decoded sentence: 21*cos(c)**2-94*cos(c)-247
Target sentence: 21*cos(c)**2-94*cos(c)-247
-
Input sentence: -8*j*(-8*j-3) 
Decoded sentence: 64*j**2+24*j
Target sentence: 64*j**2+24*j


# Validation for 10K samples

In [ ]:
def predict(factors: str):
    encoder_input_data_val = np.zeros( (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    for t, char in enumerate(factors):
        encoder_input_data_val[0, t, input_token_index[char]] = 1.0
    encoder_input_data_val[0, t + 1 :, input_token_index[" "]] = 1.0
    decoded_sentence = decode_sequence(encoder_input_data_val)
    return decoded_sentence.strip()

In [ ]:
count = 0
total = 0
for text in text_pairs[min(num_samples, len(text_pairs) - 1):]:
    total += 1
    if predict(text[0]) == text[1]:
        count += 1
    if total % 10_000 == 0:
        print("Current Samples : ", total)
        print("Current Accuracy : ", count / total)

print("Val Accuracy : ",count / total)

Current Samples :  10000
Current Accuracy :  0.89


In [ ]:
import sys
import numpy as np
import urllib.request
from typing import Tuple

MAX_SEQUENCE_LENGTH = 29
#TRAIN_URL = "https://scale-static-assets.s3-us-west-2.amazonaws.com/ml-interview/expand/train.txt"
TRAIN_URL = "https://scale-static-assets.s3-us-west-2.amazonaws.com/ml-interview/expand/train.txt"


def load_file(file_path: str) -> Tuple[Tuple[str], Tuple[str]]:
    """ A helper functions that loads the file into a tuple of strings

    :param file_path: path to the data file
    :return factors: (LHS) inputs to the model
            expansions: (RHS) group truth
    """
    with urllib.request.urlopen(file_path) as url:
        data = url.readlines()
        factors, expansions = zip(*[line.decode('utf8').strip().split("=") for line in data])
        return factors, expansions


def score(true_expansion: str, pred_expansion: str) -> int:
    """ the scoring function - this is how the model will be evaluated

    :param true_expansion: group truth string
    :param pred_expansion: predicted string
    :return:
    """
    return int(true_expansion == pred_expansion)


# --------- START OF IMPLEMENT THIS --------- #
def predict(factors: str):
    encoder_input_data_val = np.zeros( (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    for t, char in enumerate(factors):
        encoder_input_data_val[0, t, input_token_index[char]] = 1.0
    encoder_input_data_val[0, t + 1 :, input_token_index[" "]] = 1.0
    decoded_sentence = decode_sequence(encoder_input_data_val)
    return decoded_sentence.strip()


# --------- END OF IMPLEMENT THIS --------- #


def main(filepath: str):
    factors, expansions = load_file(filepath)
    pred = [predict(f) for f in factors]
    scores = [score(te, pe) for te, pe in zip(expansions, pred)]
    print(np.mean(scores))


if __name__ == "__main__":
    main("test.txt" if "-t" in sys.argv else TRAIN_URL)

In [10]:
class Model_Vars():
    def __init__(self, latent_dim, input_token_index, target_token_index,
                 num_decoder_tokens, max_decoder_seq_length, max_encoder_seq_length, num_encoder_tokens):
        self.latent_dim = latent_dim
        self.input_token_index = input_token_index
        self.target_token_index = target_token_index
        self.num_decoder_tokens = num_decoder_tokens
        self.max_decoder_seq_length = max_decoder_seq_length
        self.max_encoder_seq_length = max_encoder_seq_length
        self.num_encoder_tokens = num_encoder_tokens
        

In [11]:
modeVars = Model_Vars(latent_dim, input_token_index, target_token_index, num_decoder_tokens, max_decoder_seq_length, max_encoder_seq_length, num_encoder_tokens)

In [17]:
import pickle
filehandler = open("model_vars.obj", 'wb') 
pickle.dump(modeVars, filehandler)
filehandler.close()